In [ ]:
%pip install -qU langgraph langchain langchain-google-genai pydantic langchain-community wikipedia grandalf

import os
import operator
from typing import Annotated, Sequence, TypedDict, List, Union, Literal

# LangChain / LangGraph Imports
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage, SystemMessage
from langchain_core.tools import tool
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import ToolNode, tools_condition

# --- CONFIGURATION ---
# Please set your Google API Key here
from google.colab import userdata
os.environ["GEMINI_API_KEY"] = userdata.get("GEMINI_API_KEY")

# Initialize our LLM (The Brain)
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0)

print("✅ System initialized. LLM ready.")

✅ System initialized. LLM ready.


## Part 1: Core Foundations 🏗️

### **Theory: Chain vs. Graph**

In traditional **Chains** (e.g., `LCEL`), execution is a Directed Acyclic Graph (DAG)—usually a straight line.


However, real-world tasks require **Cycles** and **State**. If Step B fails, we might want to go back to Step A. If the data is ambiguous, we might want to loop until it is clarified.

**LangGraph** introduces a graph structure where:

1. **Nodes:** Agents or functions that perform work.
2. **Edges:** Control flow rules (go to next node, or branch based on condition).
3. **State:** A shared data structure accessed and updated by nodes.

### **The "Hello World" of Graphs**

Let's build a simple linear graph to understand the syntax.


In [ ]:
# 1. Define the State (The memory of our graph)
class SimpleState(TypedDict):
    task: str
    result: str

# 2. Define Nodes (The workers)
def research_node(state: SimpleState):
    print(f"🔎 Researching: {state['task']}")
    # Simulating work
    return {"result": f"Found technical docs for {state['task']}"}

def format_node(state: SimpleState):
    print("📝 Formatting report...")
    return {"result": f"{state['result']} -> Formatted as Markdown."}

# 3. Build the Graph
workflow = StateGraph(SimpleState)

# Add nodes
workflow.add_node("researcher", research_node)
workflow.add_node("formatter", format_node)

# Add edges (Linear flow: Start -> Researcher -> Formatter -> End)
workflow.add_edge(START, "researcher")
workflow.add_edge("researcher", "formatter")
workflow.add_edge("formatter", END)

# 4. Compile
app = workflow.compile()

# 5. Execute
output = app.invoke({"task": "LangGraph Basics"})
print(f"\n🚀 Final Output: {output['result']}")

🔎 Researching: LangGraph Basics
📝 Formatting report...

🚀 Final Output: Found technical docs for LangGraph Basics -> Formatted as Markdown.


## Part 2: State Management 🧠

### **Theory: The Shared Brain**

The `State` is the most critical concept in LangGraph. Unlike chains where data is passed like a hot potato, in LangGraph, nodes **read from** and **write to** a shared schema.

* **Immutable Updates:** By default, when a node returns `{"key": "value"}`, it overwrites the existing key.
* **Reducers (Annotated):** Sometimes we want to *append* data (like a chat history) rather than overwrite it. We use `Annotated` with an operator (usually `operator.add`) to achieve this.

Let's upgrade our Research Assistant's brain to handle chat history and multiple search results.


In [ ]:
# Defining a Robust State for our Assistant
class ResearchState(TypedDict):
    # 'operator.add' ensures that when a node returns new messages,
    # they are APPENDED to the list, not overwriting it.
    messages: Annotated[list[BaseMessage], operator.add]

    # These fields will be overwritten by the latest update
    query: str
    documents: List[str]
    draft_report: str
    critique_count: int

print("✅ ResearchState defined with append-only message history.")


✅ ResearchState defined with append-only message history.



## Part 3: Nodes & Execution 🏃‍♂️

### **Theory: Nodes as Functions**

A Node is simply a Python function that:

1. **Receives** the current `State`.
2. **Performs** logic (calling an LLM, searching, calculating).
3. **Returns** a dictionary of updates to apply to the `State`.

We will create two functional nodes for our assistant:

1. **Search Node:** Simulates fetching data (to keep this notebook robust without external API dependencies, we will mock the search logic, but it uses real data structures).
2. **Curator Node:** An LLM that summarizes the search results.


In [ ]:
def search_node(state: ResearchState):
    """
    Simulates a search engine. In production, this would call Tavily or Google Search.
    """
    query = state["query"]
    print(f"🌐 Search Agent: Searching for '{query}'...")

    # Simulated search results based on query context
    simulated_docs = [
        f"Doc A: Technical overview of {query}.",
        f"Doc B: Implementation details for {query} using Python.",
        f"Doc C: Common pitfalls when using {query}."
    ]

    # We return ONLY the field we want to update
    return {"documents": simulated_docs}

def curator_node(state: ResearchState):
    """
    Uses the LLM to summarize the found documents.
    """
    print("🧠 Curator Agent: Synthesizing information...")
    docs = "\n".join(state["documents"])

    prompt = f"""
    You are a senior technical writer.
    Summarize the following documents regarding '{state['query']}':

    {docs}
    """

    response = llm.invoke(prompt)

    # Update the draft_report and add the AI message to history
    return {
        "draft_report": response.content,
        "messages": [response]
    }

print("✅ Nodes defined.")

✅ Nodes defined.


## Part 4: Edges & Control Flow 🔀

### **Theory: Conditional Edges (Routers)**

Static edges (`add_edge("a", "b")`) are deterministic.
**Conditional Edges** allow the graph to choose the next step dynamically based on the state.

We will implement a **Router** that analyzes the user's intent.

* If the user asks for code  Go to Coder.
* If the user asks for concepts  Go to Researcher.


In [ ]:
# 1. Define the Router Logic
def intent_router(state: ResearchState) -> Literal["search_node", "coder_node"]:
    """
    Classifies the user's intent to route to the correct node.
    """
    last_message = state["messages"][-1].content

    # We ask the LLM to classify
    classification_prompt = f"""
    Classify the following user request into one of two categories: 'CODING' or 'RESEARCH'.

    Request: {last_message}

    Return ONLY the word 'CODING' or 'RESEARCH'.
    """

    response = llm.invoke(classification_prompt).content.strip().upper()

    if "CODING" in response:
        print("🔀 Router: Detected Coding Request -> Routing to Coder.")
        return "coder_node"
    else:
        print("🔀 Router: Detected Research Request -> Routing to Search.")
        return "search_node"

# 2. Define a dummy Coder Node for the router target
def coder_node(state: ResearchState):
    print("💻 Coder Agent: Generating Python code snippet...")
    return {"draft_report": f"```python\n# Code for {state['query']}\nprint('Hello World')\n```"}

print("✅ Router and Coder Node defined.")

✅ Router and Coder Node defined.


## Part 5: Compilation & Execution ⚙️

### **Theory: The StateGraph**

We now assemble the pieces.

1. Initialize `StateGraph(ResearchState)`.
2. Add Nodes.
3. Add **Conditional Edges** using the router.
4. `compile()` the graph into a `Runnable`.


In [ ]:
workflow_routing = StateGraph(ResearchState)

# Add Nodes
workflow_routing.add_node("search_node", search_node)
workflow_routing.add_node("curator_node", curator_node)
workflow_routing.add_node("coder_node", coder_node)

# Add conditional edges from START using intent_router
workflow_routing.add_conditional_edges(
    START,
    intent_router,
    {
        "search_node": "search_node",
        "coder_node": "coder_node",
    }
)

# Let's attach the Curator after search
workflow_routing.add_edge("search_node", "curator_node")
workflow_routing.add_edge("curator_node", END)
workflow_routing.add_edge("coder_node", END)

# Compile
app_routing = workflow_routing.compile()

# Visualize
print(app_routing.get_graph().draw_ascii())

# Test Execution
print("\n--- 🧪 Execution Test ---")
inputs = {
    "query": "LangGraph state management",
    "messages": [HumanMessage(content="Explain LangGraph state.")]
}
result = app_routing.invoke(inputs)
print(f"\n📄 Final Report:\n{result['draft_report']}")

print("\n--- 🧪 Execution Test (Coding Request) ---")
inputs_code = {
    "query": "Python code for factorial",
    "messages": [HumanMessage(content="Give me Python code for calculating factorial.")]
}
result_code = app_routing.invoke(inputs_code)
print(f"\n📄 Final Report (Code):\n{result_code['draft_report']}")

              +-----------+               
              | __start__ |               
              +-----------+               
              ..           ..             
            ..               ..           
          ..                   ..         
+-------------+                  ..       
| search_node |                   .       
+-------------+                   .       
        *                         .       
        *                         .       
        *                         .       
+--------------+           +------------+ 
| curator_node |           | coder_node | 
+--------------+           +------------+ 
              **           **             
                **       **               
                  **   **                 
                +---------+               
                | __end__ |               
                +---------+               

--- 🧪 Execution Test ---
🔀 Router: Detected Coding Request -> Routing to Coder.
💻 Coder Agent: G

## Part 6: Loops & Self-Correction 🔄

### **Theory: Reflection**

One-shot generation often leads to hallucinations or poor quality.
**Agentic Workflow Pattern:** .

We will add a **Critique Node**.

1. LLM reviews the `draft_report`.
2. If the quality is low, it returns `REJECT` and we loop back to the generator.
3. If high, it returns `APPROVE` and we go to END.

*Crucial:* We must track `critique_count` in our state to prevent infinite loops!


In [ ]:
def critique_node(state: ResearchState):
    """
    Reviews the draft report.
    """
    print("🧐 Critic Agent: Reviewing draft...")
    current_draft = state["draft_report"]
    current_count = state.get("critique_count", 0)

    # Safety Valve: Prevent infinite loops
    if current_count >= 3:
        print("⚠️ Critic: Max retries reached. Approving anyway.")
        # The should_continue function will handle the END transition based on this count
        return {"critique_count": current_count + 1}

    # Ask LLM to critique
    prompt = f"""
    Review this draft for technical accuracy, brevity, and completeness:
    {current_draft}

    If the draft is excellent and requires no further improvements, say 'APPROVE'.
    Otherwise, provide specific, concise feedback on how to improve it.
    """
    response = llm.invoke(prompt).content

    # We store the critique as a message to give context to the writer next time
    return {
        "critique_count": current_count + 1,
        "messages": [AIMessage(content=f"Critique: {response}")]
    }

def should_continue(state: ResearchState) -> Literal["curator_node", "end_node"]:
    """
    Decides whether to loop back or end based on the last message.
    """
    last_message_content = state["messages"][-1].content

    # Check if the critique explicitly approved or if max retries are met
    if "APPROVE" in last_message_content or state["critique_count"] > 2:
        print("✅ Critic: Quality standard met.")
        return "end_node"
    else:
        print("🔙 Critic: Rejection. Looping back to Curator.")
        return "curator_node"

# Update Graph with Cycle
workflow_reflection = StateGraph(ResearchState)

workflow_reflection.add_node("search_node", search_node)
workflow_reflection.add_node("curator_node", curator_node)
workflow_reflection.add_node("critique_node", critique_node)

workflow_reflection.add_edge(START, "search_node")
workflow_reflection.add_edge("search_node", "curator_node")
workflow_reflection.add_edge("curator_node", "critique_node")

# The Conditional Edge creates the Cycle
workflow_reflection.add_conditional_edges(
    "critique_node",
    should_continue,
    {
        "end_node": END,
        "curator_node": "curator_node"
    }
)

app_reflection = workflow_reflection.compile()
print(app_reflection.get_graph().draw_ascii())

# --- Small Test for Reflection Graph ---
print("\n--- 🧪 Reflection Graph Test ---")
test_input_reflection = {
    "query": "Quantum Computing",
    "messages": [HumanMessage(content="Generate a short report on quantum computing.")],
    "critique_count": 0
}

final_reflection_output = app_reflection.invoke(test_input_reflection)
print(f"\n📝 Final Report (after critique loop):\n{final_reflection_output['draft_report']}")
print(f"Total critiques: {final_reflection_output['critique_count']}")

  +-----------+    
  | __start__ |    
  +-----------+    
        *          
        *          
        *          
 +-------------+   
 | search_node |   
 +-------------+   
        *          
        *          
        *          
+--------------+   
| curator_node |   
+--------------+   
        *          
        *          
        *          
+---------------+  
| critique_node |  
+---------------+  
        .          
        .          
        .          
   +---------+     
   | __end__ |     
   +---------+     

--- 🧪 Reflection Graph Test ---
🌐 Search Agent: Searching for 'Quantum Computing'...
🧠 Curator Agent: Synthesizing information...
🧐 Critic Agent: Reviewing draft...
✅ Critic: Quality standard met.

📝 Final Report (after critique loop):
As a senior technical writer, I've reviewed the provided document outlines on Quantum Computing. Here's a concise summary of their key contributions:

---

### Summary of Quantum Computing Documents

This summary synthesize

## Part 7: Tools & Actions 🛠️

### **Theory: ToolNode**

LLMs can't do math or access real-time data natively. We bind **Tools** to the LLM.
When the LLM decides to call a tool, it outputs a tool call request. A `ToolNode` executes this request and feeds the result back.

We will integrate `Wikipedia` and a `Calculator`.


In [ ]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

# 1. Define Tools
@tool
def calculator(expression: str) -> str:
    """Calculates a mathematical expression."""
    try:
        return f"{expression} = {eval(expression)}"
    except:
        return "Invalid syntax"

# Setup Wikipedia Tool
wiki = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())

tools = [calculator, wiki]

# 2. Bind Tools to LLM
# This tells the LLM "You have these functions available"
llm_with_tools = llm.bind_tools(tools)

# 3. Create a Node that uses the Tool-bound LLM
def agent_node(state: ResearchState):
    print("🤖 Agent: Reasoning...")
    messages = state["messages"]
    response = llm_with_tools.invoke(messages)
    return {"messages": [response]}

# 4. Create the ToolNode (Pre-built by LangGraph to execute tool calls)
tool_node = ToolNode(tools)

# 5. Build Graph
tool_graph = StateGraph(ResearchState)
tool_graph.add_node("agent", agent_node)
tool_graph.add_node("tools", tool_node)

tool_graph.add_edge(START, "agent")

# conditional_edges logic:
# If LLM response has tool_calls -> Go to 'tools'
# If LLM response is text -> Go to END
tool_graph.add_conditional_edges("agent", tools_condition)
tool_graph.add_edge("tools", "agent") # Loop back to agent to read tool output

app_tools = tool_graph.compile()

print("✅ Tool Graph built.")

# --- Small Test for Tool Graph ---
print("\n--- 🧪 Tool Graph Test ---")

# Test 1: Calculator tool
print("\n--- Testing Calculator Tool ---")
calculator_input = {
    "messages": [HumanMessage(content="What is 123 + 456?")]
}
calculator_result = app_tools.invoke(calculator_input)
print(f"Calculator Result: {calculator_result['messages'][-1].content}")

# Test 2: Wikipedia tool
print("\n--- Testing Wikipedia Tool ---")
wikipedia_input = {
    "messages": [HumanMessage(content="Who is the current CEO of Google?")]
}
wikipedia_result = app_tools.invoke(wikipedia_input)
print(f"Wikipedia Result: {wikipedia_result['messages'][-1].content}")

✅ Tool Graph built.

--- 🧪 Tool Graph Test ---

--- Testing Calculator Tool ---
🤖 Agent: Reasoning...
🤖 Agent: Reasoning...
Calculator Result: The sum of 123 and 456 is 579.

--- Testing Wikipedia Tool ---
🤖 Agent: Reasoning...
🤖 Agent: Reasoning...
Wikipedia Result: [{'type': 'text', 'text': 'Sundar Pichai is the current CEO of Google.', 'extras': {'signature': 'CtcEAXLI2nwjAlm6VBuYJHz4Hca6V+Br1s831iUq5Hj033Y+ZvLhqOKIkEG21T83KO37uXfRJ/dvvR6s6LDuI5yIgTst36AvZrRMDH57JA22ucV1BcOveZzSHf+iAZmlHl6X0BL/B9Xe2ZU1AyP3/UV60Q/5TjTtVs+kVq+T2OjTv5Nh15BLj/OkCJsk+I4HhohfR5lRnrMUiPZn9AH27gxZ/fwCzeW6kw+z2TlUy32S2nNqqG+OagqPj5XiGnb6lUUbvjK70vY2bgAbd3d36ro148oG+tX4n76ndGvhEpCjg0pJvvMgOTJgkj/Z/NUEwL4ifDxLthjtGlK6wMidkTNlE80sa5uadoKsO8l/ZtfX9WyWCa8u7u6lA8Qj1Tfwbu5Xffhma+HpDPyMSBGRkeTOYrsYFMylNL8z4f5/vkSipniv3vfv1GaQuq0gWA2WqjX//N7q5IstxISSAgb2WpVb+Go2F4U6m0wuCX4gmpwYloMesAPWbmJY+UbTKgN/cshKTZCCBg/QgqOChhV3ayQk5I7Pqdhx9nr8pA/EpDdYYn7xDv3sg9lzZlfChN9ZaOsqssWdYyYt/7QoUQoyVhnxIL8dgD38gYIvpawCAqEL++NZUNB4vcnPA2

## Part 8: Time Travel with Checkpointers 🕰️

### **Theory: Persistence**

Normally, when a script ends, memory is lost.
**Checkpointers** save the `State` to a database (or in-memory for testing) at every step.
This allows:

1. **Human-in-the-loop:** Pause, wait for human approval, then resume.
2. **Error Recovery:** Retry from the last known good state.


In [ ]:
from langgraph.checkpoint.memory import MemorySaver

# 1. Initialize Memory
memory = MemorySaver()

# 2. Compile with Checkpointer
# We reuse our reflection graph from Part 6
app_persistent = workflow_reflection.compile(checkpointer=memory)

# 3. Define a Thread ID (Like a session ID)
thread_config = {"configurable": {"thread_id": "session_1"}}

# 4. Run Step 1 - Initial Research
print("--- 🟢 Run 1: Initial Research (Streaming Events) ---")
input_1 = {
    "query": "LangGraph Persistence",
    "messages": [HumanMessage(content="Research persistence in LangGraph.")],
    "critique_count": 0 # Reset critique count for a fresh run
}

print("\n--- Events from Run 1 ---")
for s in app_persistent.stream(input_1, thread_config):
    print(s)
    # After each step, inspect the state to show persistence
    current_state = app_persistent.get_state(thread_config)
    print(f"  📸 Current State after step: {current_state.values.get('query')}, Critiques: {current_state.values.get('critique_count')}")

state_snapshot_1 = app_persistent.get_state(thread_config)
print(f"\n📸 Final Snapshot of State after Run 1: Query='{state_snapshot_1.values['query']}', Critiques='{state_snapshot_1.values['critique_count']}'")


# 5. Resume / Continue Conversation (Run 2) - This uses the saved state from Run 1
print("\n--- 🟢 Run 2: Follow up (Retains History and State) ---")
input_2 = {"messages": [HumanMessage(content="Summarize the persistence findings in 5 words.")]}

print("\n--- Events from Run 2 ---")
# Because we pass the same thread_id, it remembers the previous context and state
for s in app_persistent.stream(input_2, thread_config):
    print(s)
    current_state = app_persistent.get_state(thread_config)
    print(f"  📸 Current State after step: {current_state.values.get('query')}, Critiques: {current_state.values.get('critique_count')}")

final_state_after_run2 = app_persistent.get_state(thread_config)
print(f"\n📝 Final Report (after follow-up):\n{final_state_after_run2.values['draft_report']}")
print(f"Total critiques after follow-up: {final_state_after_run2.values['critique_count']}")

--- 🟢 Run 1: Initial Research (Streaming Events) ---

--- Events from Run 1 ---
🌐 Search Agent: Searching for 'LangGraph Persistence'...
{'search_node': {'documents': ['Doc A: Technical overview of LangGraph Persistence.', 'Doc B: Implementation details for LangGraph Persistence using Python.', 'Doc C: Common pitfalls when using LangGraph Persistence.']}}
  📸 Current State after step: LangGraph Persistence, Critiques: 0
🧠 Curator Agent: Synthesizing information...
{'curator_node': {'draft_report': 'As a senior technical writer, I\'ve reviewed the hypothetical content of these three documents concerning LangGraph Persistence. Here is a consolidated summary:\n\n---\n\n### LangGraph Persistence: A Comprehensive Overview\n\nLangGraph Persistence is a fundamental capability designed to maintain and manage the state of complex, multi-step agentic workflows within the LangGraph framework. It enables agents to remember conversational context, intermediate reasoning steps, and tool interactions